In [ ]:
import json
import time
import pickle
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import random
from unidecode import unidecode
import sys
sys.setrecursionlimit(10000)
import networkx as nx
import matplotlib.pyplot as plt

### 1. Obtendo lista de gêneros, artistas e músicas

In [ ]:
# Gêneros (todos)
r = requests.get('https://www.letras.com.br/estilos')
soup = BeautifulSoup(r.text)
generos = [k.get("href").replace("https://www.letras.com.br/estilos/", "") for k in soup.find_all("a") if "/estilos/" in k.get("href")]

# Selecionando os gêneros principais
generos_ =  ['axe',
             'blues',
             'bossa-nova',
             'brega',
             'forro',
             'funk',
             'gospel',
             'hard-rock',
             'heavy-metal',
             'hip-hop-rap',
             'infantil',
             'jazz',
             'jovem-guarda',
             'mpb',
             'pagode',
             'pop',
             'poprock',
             'post-rock',
             'punk-rock',
             'reggae',
             'regional',
             'rock-roll',
             'romantico',
             'samba',
             'samba-enredo',
             'sertanejo',
             'soul',
             'velha-guarda']

In [ ]:
# Artistas

def pega_lista_artistas(genero):
    r = requests.get("https://www.letras.com.br/estilos/"+genero)
    soup = BeautifulSoup(r.text)
    lista_artistas = list(set([k.get("href").split("%")[0].replace("https://www.letras.com.br/", "") for k in soup.find_all("section", class_="pb-0 pb-lg-5")[-1].find("ul", class_="row no-gutters").find_all("a")]))
    return lista_artistas

artistas_por_genero = {genero: pega_lista_artistas(genero) for genero in generos_}
artistas = list(np.unique(np.array(sum(artistas_por_genero.values(), []))))


In [ ]:
len(artistas)

In [ ]:
# Músicas
def pega_lista_musicas(artista):
    r = requests.get('https://www.letras.com.br/' + artista)
    soup = BeautifulSoup(r.text)
    lista_musicas = [k.get('href').replace('https://www.letras.com.br/'+artista+'/', '') for k in soup.find('ul', class_ = 'row').find_all('a')]
    return lista_musicas

In [ ]:
musicas_por_artista = {}
erros = []
count = 0
for artista in artistas:
    count += 1
    try:
        musicas_por_artista[artista] = pega_lista_musicas(artista)
    except:
        erros.append(artista)
        pass
    if count % 100 == 0:
        print(count, count/len(artistas), len(erros))

In [ ]:
musicas = list(np.unique(np.array(sum(musicas_por_artista.values(), []))))
len(musicas)

### 2. Compositores

##### 2.1. Baixando dados 

In [ ]:
def busca_compositor_letras(url):
    page = requests.get(url)
    html_doc = page.text
    soup = BeautifulSoup(html_doc, 'html.parser')

    try:
        compositor = soup.find(class_="mt-4 pt-4 lyrics-comp").find('span').string
        compositor
    except:
        compositor = 'sem_info'
    return compositor

In [ ]:
lista_final = []
#count_art = 0

In [ ]:
for artista in artistas[10000:]:
    try:
        count_mus = 0
        lista_mus_art = musicas_por_artista[artista]

        for musica in lista_mus_art:
            try:
                link = 'https://www.letras.com.br/'+artista+'/'+musica
                composicao = busca_compositor_letras(link)
                lista_final.append([artista, musica, composicao])
                count_mus+=1
            except:
                try:
                    link = 'https://www.letras.com.br/'+artista+'/'+musica
                    composicao = busca_compositor_letras(link)
                    lista_final.append([artista, musica, composicao])
                    count_mus+=1
                except:
                    try:
                        link = 'https://www.letras.com.br/'+artista+'/'+musica
                        composicao = busca_compositor_letras(link)
                        lista_final.append([artista, musica, composicao])
                        count_mus+=1
                    except Exception as error:
                        print(error, musica, artista)
                        pass

        print(artista, count_art, count_art/len(artistas))
    except:
        pass
    count_art+=1

In [ ]:
# Gravando em 11 partes pois é muito pesado
with open('lista_letras_d11.pkl', 'wb') as f:
    pickle.dump(lista_final, f)

##### 2.2. Tratando dados

In [ ]:
def ajuste_texto(texto):
    for i in [' e ', '&', ',', '/', ' - ']:
        texto = texto.replace(i, '|')
        res = [unidecode(k).lower().strip().replace(' ', '-') for k in texto.split('|')]
    return res


In [ ]:
for i in ['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11']:

    dict_grafo = {'ent_a': [], 'ent_b':[], 'desc': []}

    with open('lista_letras_'+i+'.pkl', 'rb') as handle:
        lista_ = pickle.load(handle)

lista_tratada = [[k[0], k[1], ajuste_texto(k[2])] for k in lista_ if k[2] not in ['sem_info', None]]

for info in lista_tratada:    
    if type(info[2])==list:
        for j in info[2]:
            dict_grafo['interprete'].append(info[0])
            dict_grafo['musica'].append(info[1])
            dict_grafo['compositor'].append(j)            
    else:
        dict_grafo['interprete'].append(info[0])
        dict_grafo['musica'].append(info[1])
        dict_grafo['compositor'].append(info[2])
        

with open('dict_grafo_'+i+'.pkl', 'wb') as f:
    pickle.dump(dict_grafo, f)
print(i)

### 3. Grafo

In [ ]:
dict_pre_grafo = {'interprete': [], 'musica':[], 'compositor': []}


In [ ]:
for i in ['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11']:
    with open('dict_grafo_'+i+'.pkl', 'rb') as handle:
        dict_ = pickle.load(handle)
    dict_pre_grafo['interprete'] += dict_['interprete']
    dict_pre_grafo['musica'] += dict_['musica']
    dict_pre_grafo['compositor'] += dict_['compositor']
    print(i)

In [ ]:
df_pre_grafo = pd.DataFrame(dict_pre_grafo)

In [ ]:
len(df_pre_grafo)

In [ ]:
df_compositores = df_pre_grafo[['compositor']].drop_duplicates().reset_index(drop=True)
df_compositores = df_compositores.reset_index()
df_compositores.head()

In [ ]:
df_pre_grafo = df_pre_grafo.merge(df_compositores, on='compositor', how = 'left')
df_pre_grafo['musica_idx'] = df_pre_grafo['musica'] + '_' + df_pre_grafo['index'].apply(lambda x: str(x))
df_pre_grafo.head()

In [ ]:
# Pequenos ajustes de compositores com nomes escritos de forma diferente

def substituicoes(nome):
    if nome=='roberto-carlos-braga':
        nome_ = 'roberto-carlos'
    elif nome =='caetano-emmanuel-viana-teles-veloso':
        nome_ = 'caetano-veloso'
    elif nome =='gilberto-passos-gil-moreira':
        nome_ = 'gilberto-gil'
    elif nome =='antonio-carlos-jobim':
        nome_ = 'tom-jobim'
    elif nome =='francisco-buarque-de-hollanda':
        nome_ = 'chico-buarque'
    elif nome =='chico-buarque-de-hollanda':
        nome_ = 'chico-buarque'
    elif nome =='luiz-gonzaga-do-nascimento':
        nome_ = 'luiz-gonzaga'
    elif nome =='djavan-caetano-viana':
        nome_ = 'djavan'
    elif nome =='luiz-gonzaga-do-nascimento-junior':
        nome_ = 'gonzaguinha'
    elif nome =='fernando-rocha-brant':
        nome_ = 'fernando-brant'
    elif nome =='angenor-de-oliveira':
        nome_ = 'cartola'
    elif nome =='erasmo-esteves':
        nome_ = 'erasmo-carlos'
    else:
        nome_ = nome
    return nome_

df_pre_grafo['interprete'] = df_pre_grafo['interprete'].map(substituicoes)
df_pre_grafo['compositor'] = df_pre_grafo['compositor'].map(substituicoes)

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(df_pre_grafo['interprete'])
G.add_nodes_from(df_pre_grafo['musica_idx'])
G.add_nodes_from(df_pre_grafo['compositor'])


# Add edges from the DataFrame
edges_comp = [(row['musica_idx'], row['compositor'], 2) for index, row in df_pre_grafo.iterrows()]
G.add_weighted_edges_from(edges_comp)

edges_grav = [(row['interprete'], row['musica_idx'], 1) for index, row in df_pre_grafo.iterrows()]
G.add_weighted_edges_from(edges_grav)

In [ ]:
rank_eig_centrality = dict(sorted(nx.eigenvector_centrality(G, weight='weight').items(), key=lambda item: item[1], reverse=True))
#rank_eig_centrality = dict(sorted(nx.eigenvector_centrality(G).items(), key=lambda item: item[1], reverse=True))

rank_eig_centrality

In [ ]:
# Vou fazer um filtro dos 2000 compositores com mais composições, para "limpar" o grafo
rank_compositores = df_pre_grafo[['compositor', 'musica_idx']].groupby(['compositor']).nunique().reset_index().sort_values('musica_idx', ascending=False).head(2000)
lista_compositores_top2000 = list(rank_compositores['compositor'])


In [ ]:
lista_rank = list(rank_eig_centrality.keys())
lista_rank_compositores = [k for k in lista_rank if k in lista_compositores_top2000]

print(lista_rank[:10])
print(lista_rank_compositores[:10])

In [ ]:
rank_eig_centrality_ = {key: rank_eig_centrality[key] for key in lista_rank_compositores}
rank_eig_centrality_

In [ ]:
list(rank_eig_centrality_.values())

In [ ]:
rank_eig_centrality_df = pd.DataFrame({'compositor':list(rank_eig_centrality_.keys()), 'nota':list(rank_eig_centrality_.values())})
rank_eig_centrality_df = rank_eig_centrality_df.reset_index()
rank_eig_centrality_df['index'] = rank_eig_centrality_df['index']+1
rank_eig_centrality_df


In [ ]:
top_20 = rank_eig_centrality_df.head(20)
top_20

In [ ]:
# Agregando informações

def agrega_info_grafo(compositor):
    df_ = df_pre_grafo[df_pre_grafo['compositor']==compositor]
    num_interpretes = len([k for k in list(set(df_['interprete'])) if k !=compositor])
    num_musicas = len(list(set(df_['musica'])))
    posicao = top_20[top_20['compositor']==compositor].iloc[0]['index']
    return f'Posição:{posicao}, Número de músicas: {num_musicas}, Número de intérpretes distintos: {num_interpretes}'

In [ ]:
top_20['info_adicional'] = top_20['compositor'].map(agrega_info_grafo)


##### Resumo artista

In [ ]:
artista = 'isolda'

df_ = df_pre_grafo[df_pre_grafo['compositor']==artista]

# Nós
lista_interpretes = [k for k in list(set(df_['interprete'])) if k !=artista]
lista_musicas = list(set(df_['musica']))

nodes = pd.concat([pd.DataFrame({'nome':lista_interpretes, 'tipo':'interprete'}), 
                   pd.DataFrame({'nome':lista_musicas, 'tipo':'musica'}),
                   pd.DataFrame({'nome':[artista], 'tipo':'compositor'})])

# Arestas
link_comp_mus = df_[['compositor', 'musica']]
link_comp_mus.columns = ['ent_a', 'ent_b']
link_mus_int = df_[['musica', 'interprete']]
link_mus_int.columns = ['ent_a', 'ent_b']
links = pd.DataFrame({'ent_a':[], 'ent_b':[]})
links = pd.concat([links, link_comp_mus, link_mus_int])
links = links[links['ent_b']!=artista]
links = links.drop_duplicates().reset_index(drop=True)
links

In [ ]:
with pd.ExcelWriter("links.xlsx") as writer:
    nodes.to_excel(writer, sheet_name="nodes", index=False)
    links.to_excel(writer, sheet_name="links", index=False)

In [ ]:
rank_eig_centrality_df[rank_eig_centrality_df['compositor']==artista]

In [ ]:
# Plotando

def plot_graph_with_two_layers(G, start_node, image_file='graph.png', pdf_file='graph.pdf'):
    # Verifica se o nó inicial está no grafo
    if start_node not in G:
        raise ValueError(f"Nó {start_node} não encontrado no grafo.")
    
    # Primeira camada: conexões que partem do nó inicial
    edges_first_layer = list(G.edges(start_node))
    
    # Obter os nós conectados na primeira camada
    first_layer_nodes = {target for _, target in edges_first_layer}
    
    # Segunda camada: conexões que partem dos nós conectados na primeira camada
    edges_second_layer = []
    for node in first_layer_nodes:
        edges_second_layer.extend(G.edges(node))

    # Combina as duas camadas de arestas
    all_edges = edges_first_layer + edges_second_layer

    # Cria o subgrafo a partir de todas as arestas
    subgraph = G.edge_subgraph(all_edges).copy()

    # Adiciona o nó inicial ao subgrafo (caso ele não tenha arestas)
    if len(edges_first_layer) == 0:
        subgraph.add_node(start_node)

    # Define a posição dos nós para exibição
    pos = nx.spring_layout(subgraph)
    
    # Desenha o subgrafo
    plt.figure(figsize=(80, 60))
    nx.draw_networkx(subgraph, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=500, font_size=12, font_color='black')
    
    # Exibe o diagrama
    plt.title(f"Conexões partindo do nó {start_node} e suas conexões")
    
     # Salva a imagem em alta qualidade (PNG)
    plt.savefig(image_file, format="png", dpi=300)
    
    # Salva o grafo em formato PDF
    plt.savefig(pdf_file, format="pdf")
    
    plt.show()

In [ ]:
plot_graph_with_two_layers(G, artista)